In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
movies_titles_dataset = pd.read_csv("Movie_Id_Titles")
movies_rating_dataset = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
movies_rating_dataset.drop(['timestamp'], axis = 1, inplace = True)
movies_rating_dataset = pd.merge(movies_rating_dataset, movies_titles_dataset, on = 'item_id') 

In [3]:
mean_data = movies_rating_dataset.groupby('title')['rating'].describe()['mean']
count_data = movies_rating_dataset.groupby('title')['rating'].describe()['count']
stats_data = pd.concat([count_data, mean_data], axis = 1)

In [4]:
userid_matrix = movies_rating_dataset.pivot_table(index = 'user_id', columns = 'title', values = 'rating')
titanic = userid_matrix['Titanic (1997)']

In [5]:
titanic_correlations = pd.DataFrame(userid_matrix.corrwith(titanic), columns=['Correlation'])
titanic_correlations = titanic_correlations.join(stats_data['count'])
titanic_correlations.dropna(inplace=True)
titanic_correlations[titanic_correlations['count']>80].sort_values('Correlation',ascending=False).head()

C:\Users\Veljko Stojanovic\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\Veljko Stojanovic\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Correlation,count
title,,
Titanic (1997),1.000000,350.0
"River Wild, The (1994)",0.497600,146.0
"Abyss, The (1989)",0.472103,151.0
Bram Stoker's Dracula (1992),0.443560,120.0
True Lies (1994),0.435104,208.0


In [6]:
movie_correlations = userid_matrix.corr(method='pearson', min_periods=80)

In [7]:
test_movies = pd.read_csv('Test_Movies.csv')
similar_movies_list = pd.Series()

for i in range(0,len(test_movies)):
    similar_movie = movie_correlations[test_movies['Movie Name'][i]].dropna()
    similar_movie = similar_movie.map(lambda x: x*test_movies['Ratings'][i])
    similar_movies_list = similar_movies_list.append(similar_movie)

C:\Users\Veljko Stojanovic\AppData\Local\Temp\ipykernel_11868\1713328996.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  similar_movies_list = pd.Series()
C:\Users\Veljko Stojanovic\AppData\Local\Temp\ipykernel_11868\1713328996.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies_list = similar_movies_list.append(similar_movie)
C:\Users\Veljko Stojanovic\AppData\Local\Temp\ipykernel_11868\1713328996.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies_list = similar_movies_list.append(similar_movie)


In [8]:
similar_movies_list.sort_values(inplace=True, ascending=False)
print(similar_movies_list.head(10))

Liar Liar (1997)                             5.000000
Con Air (1997)                               2.349141
Pretty Woman (1990)                          2.348951
Michael (1996)                               2.210110
Indiana Jones and the Last Crusade (1989)    2.072136
Top Gun (1986)                               2.028602
G.I. Jane (1997)                             1.989656
Multiplicity (1996)                          1.984302
Grumpier Old Men (1995)                      1.953494
Ghost and the Darkness, The (1996)           1.895376
dtype: float64
